Import des librairies

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
torch.set_printoptions(edgeitems=2) #Affiche les 2 premiers / derniers items d'un tensor

In [ ]:
#On va essayer de transformer des températures de farheineiht en degrés
#Tensor Features (11 exemple, avec une seule valeur)
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4])
#Tensor des labels 
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0])

t_un = 0.1 * t_u #Normalisation simple des features ()
t_un

In [ ]:
# Fonction Hypothèse
def model(t_u, w, b):
    return w * t_u + b

In [ ]:
#Fonction de coût (Mean Square Error)
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [ ]:
#Initialisation du Tensor des paramètres, en indiquant qu'il faudra calculer les gradients
params = torch.tensor([1.0, 0.0], requires_grad=True)
# Le gradient des paramètres n'existe pas encore
print(params.grad)

In [ ]:
# Calcule la Fonction de coût en lui passant le Tensor des params:
loss = loss_fn(model(t_un, *params), t_c)
# Lance la gradient Descent (Backpropagation). Pytorch a retenu les opérations réalisées pour calculer loss
loss.backward()
#Le gradient des paramètres est maintenant renseigné
params.grad

In [ ]:
# !!! Il faut remettre le gradient à 0 entre chaque itération, car les gradients s'accumulent
if params.grad is not None:
    params.grad.zero_()
print(params.grad)

In [ ]:
#Apprentissage
n_epochs=5000
learning_rate = 1e-2

for epoch in range(1, n_epochs + 1):
    if params.grad is not None:  # <1>
        params.grad.zero_()
    
    t_p = model(t_un, *params) 
    loss = loss_fn(t_p, t_c)
    loss.backward()
    
    with torch.no_grad():  # <2>
        params -= learning_rate * params.grad

    if epoch % 500 == 0:
        print('Epoch %d, Loss %f' % (epoch, float(loss)))
        
print(params)

In [ ]:
t_p = model(t_un, *params)

plt.xlabel("Temperature (°Fahrenheit)")
plt.ylabel("Temperature (°Celsius)")

# Il faut appliquer  detach à tp pour qu'il ne fasse pas partie du calcul du gradient
plt.plot(t_u.numpy(), t_p.detach().numpy())
plt.plot(t_u.numpy(), t_c.numpy(), 'o')
plt.show()